In [1]:
import json
import os
import pandas as pd
import numpy as np
path = os.getcwd()
from classes.Block import Block #the block class

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
stop_words = set(stopwords.words('english'))
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\MONSTER\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [2]:
#TODO (put done if it is done)

#variables to create:
#all tokens of e1: val_e1_tokens ([str]) DONE
#all tokens of e1: val_e2_tokens [str] DONE
#Tokens in each entityProfile of E1 ({ "eP": ['token1', 'token2',...'tokenN']}): dict_E1 DONE
#Tokens in each entityProfile of E2 ({ "eP": ['token1', 'token2',...'tokenN']}): dict_E2 DONE
#sets of token set(tokenE1 + tokenE2): commonToken DONE
#Entities of E1 that contains a ct ∈ commonToken { "cT": [1,2,3,10,....]}: e1_dict_token DONE
#Entities of E2 that contains a ct ∈ commonToken { "cT": [101,102,103,110,....]}: e2_dict_token DONE
#Attributes of E1: all_att_e1 DONE
#Attributes of E2: all_att_e2 DONE


#make a class for the block: DONE
#create the class in a seperate python file/module (?) DONE
#Block needs a: name, ib1 (innerBlock1), ib2 (innerBlock2). if one of the ib ∈ IB is empty, a block
#of a token should not be created
#in addition, cardinality and block size


#stemming and cleaning:
#Do it at stem clean function
# remove stop words DONE
# stemming
# tokenize DONE
# DISCUSS: ignore numbers, so take only words as token, not integer
#AND LEMMATIZATION
#combine lowering all letters before making the token in one stem_clean function 
#so it will be easier to implement any semantic treatment
#delete period signs

#visualizatipo

In [2]:
# Opening JSON file
f1 = open(f'{path}/data/entityCol1.json')
f2 = open(f'{path}/data/entityCol2.json')
 
# returns JSON object as
# a dictionary
ec1 = json.load(f1)
ec2 = json.load(f2)

In [3]:
def stem_clean(arr):
    joint_words = ' '.join(arr)
    joint_words = str(joint_words).lower()
    val = word_tokenize(joint_words)
    valTok = list(set([w for w in val if not w.lower() in stop_words]))
    return valTok
    

In [10]:
#get all unique key and val on both EC
all_att_e1 = []
all_val_e1 = []
dict_E1 = {}
x = 1
for d in ec1:
    values_EP = []
    for key, val in (d.items()):
        all_att_e1.append(key) #allKey
        treatedVal = str(val).lower()
        values_EP.append(treatedVal)
        all_val_e1.append(treatedVal) #allVal - treat number as string
    values_EP = stem_clean(values_EP)
    dict_E1[x] = values_EP
    x = x+1

attE1Keys = list(set(all_att_e1))
val_e1_tokens = list(set(stem_clean(all_val_e1)))


all_at_e2 = [] 
all_val_e2 = []
dict_E2 = {}
x = 101
for d in ec2:
    values_EP = []
    for key, val in (d.items()):
        all_at_e2.append(key) #allKey
        treatedVal = str(val).lower()
        values_EP.append(treatedVal)
        all_val_e2.append(treatedVal) #allVal - treat number as string
    values_EP = stem_clean(values_EP)
    dict_E2[x] = values_EP
    x = x+1
attE2Keys = list(set(all_at_e2))
val_e2_tokens = list(set(stem_clean(all_val_e2)))


#common token
#Only take token that exists in both 


<h3>Token Blocking </h3>

In [5]:
#common token
#Only take token that exists in both 
common_token = set(val_e1_tokens) & set(val_e2_tokens)

In [6]:
#from the common token create find all entity profile
#that has contains the token
e1_dict_token = {}
for t in common_token:
    x = [ ep for ep in dict_E1 if t in dict_E1[ep]]
    e1_dict_token[t] = x
    
e2_dict_token = {}
for t in common_token:
    x = [ ep for ep in dict_E2 if t in dict_E2[ep]]
    e2_dict_token[t] = x

In [7]:
all_blocks = []
for ct in common_token:
    new_block = Block(ct,e1_dict_token[ct], e2_dict_token[ct] )
    all_blocks.append(new_block)
    
all_blocks.sort(key=lambda x: x.b_cardinality, reverse=True) #sort based on its cardinality

In [8]:
#DataFrame for purpose Visualization Purpose
token_blocks_df = pd.DataFrame([x.as_dict() for x in all_blocks])
token_blocks_df.head(20)

AttributeError: 'Block' object has no attribute 'b_name'

<h3>Attribute Clustering</h3>

In [33]:
"""
It's not finished yet, please do not proceed, let me finish it on Monday. :D
"""

#Group based on the attribute for E1
#Group based on the attribute for E2
"""attE1Keys.remove('id')
attE2Keys.remove('id')
attE2Keys.remove('age')"""

attribute_blocks_ec1 = {}
attribute_blocks_ec2 = {}
for attribute in attE1Keys:
    attribute_blocks_ec1[attribute] = stem_clean(word_tokenize(' '.join(set([att[attribute].lower() for att in ec1 if attribute in att.keys()]))))
for attribute in attE2Keys:
    attribute_blocks_ec2[attribute] = stem_clean(word_tokenize(' '.join(set([att[attribute].lower() for att in ec2 if attribute in att.keys()]))))

#Similarity Calculation

def jaccard_set(list1, list2):
    """Define Jaccard Similarity function for two sets"""
    intersection = len(list(set(list1).intersection(list2)))
    union = (len(list1) + len(list2)) - intersection
    return float(intersection) / union

sim_ec1_ec2 = {}
for each in attribute_blocks_ec1:
    temp = []
    for every in attribute_blocks_ec2:
        similarity = jaccard_set(attribute_blocks_ec1[each], attribute_blocks_ec2[every])
        if similarity > 0:
            temp.append(every)
    sim_ec1_ec2[each] = temp

sim_ec2_ec1 = {}
for each in attribute_blocks_ec2:
    temp = []
    for every in attribute_blocks_ec1:
        similarity = jaccard_set(attribute_blocks_ec2[each], attribute_blocks_ec1[every])
        if similarity > 0:
            temp.append(every)
    sim_ec2_ec1[each] = temp

#Attribute Blocking


#testa = jaccard_set(attribute_blocks_ec1['domicile'], attribute_blocks_ec2['country'])